# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링

### 문제

1. 인기 리뷰어라면 핫플레이스만 다녀서 평범한 음식점에 대한 정보는 못얻지 않을까?

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

c:\Users\hare0\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2
https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2
https://m.place.naver.com/my/5bd2f3682db9d1bdd38eedca/review?v=2
https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2
https://m.place.naver.com/my/review/65bda752011aacbd86f5ef9e?v=2

In [2]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())

# 주소 이동 (SUNsDay 나 아)
url = 'https://m.place.naver.com/my/5c2b07cbc98dc4c88b3ac001/review?v=2'
driver.get(url)
time.sleep(1)


In [4]:
# 저장 
User_df = pd.DataFrame()
time.sleep(3)  

# 맨 처음 게시물 클릭
driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[5]/div/ul/li[1]/button').click()

